In [25]:
import sys
import pathlib
import os
from subprocess import call
sys.path.append("..") # relative path to module toolkit

In [2]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras_preprocessing.image import ImageDataGenerator
from toolkit import getLabelsFromDir, plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

/home/tofa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/tofa/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# setup
batch_size = 3
validation_steps = int(3454/batch_size) + 1

val_dir = "../../images/images_genus/val/"

labels = getLabelsFromDir(val_dir)

val_datagen = ImageDataGenerator(rescale=1./255.)

In [28]:
model = load_model('restnet50frozen.h5')

In [38]:
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(224, 224),
                                                    shuffle=False,
                                                    seed=1,
                                                    save_to_dir="val_images",
                                                    save_format="png")

Found 3454 images belonging to 44 classes.


In [40]:
val_generator.next()

(array([[[[0.909804  , 0.8470589 , 0.85098046],
          [0.92549026, 0.86274517, 0.86666673],
          [0.9333334 , 0.86274517, 0.86274517],
          ...,
          [0.9294118 , 0.8588236 , 0.85098046],
          [0.9294118 , 0.8588236 , 0.85098046],
          [0.92549026, 0.86274517, 0.86274517]],
 
         [[0.909804  , 0.8470589 , 0.85098046],
          [0.92549026, 0.86274517, 0.86666673],
          [0.9333334 , 0.86274517, 0.86274517],
          ...,
          [0.9294118 , 0.8588236 , 0.85098046],
          [0.9294118 , 0.8588236 , 0.85098046],
          [0.92549026, 0.86274517, 0.86274517]],
 
         [[0.9215687 , 0.8588236 , 0.86274517],
          [0.9176471 , 0.8470589 , 0.8470589 ],
          [0.92549026, 0.854902  , 0.854902  ],
          ...,
          [0.9294118 , 0.8588236 , 0.85098046],
          [0.9333334 , 0.86274517, 0.854902  ],
          [0.9215687 , 0.8588236 , 0.8588236 ]],
 
         ...,
 
         [[0.9176471 , 0.83921576, 0.8352942 ],
          [0.94509

In [20]:
val_generator.filenames

['Othius/Othius angustus_10.jpg',
 'Othius/Othius angustus_11.jpg',
 'Othius/Othius angustus_12.jpg',
 'Othius/Othius angustus_13.jpg',
 'Othius/Othius angustus_14.jpg',
 'Othius/Othius angustus_15.jpg',
 'Othius/Othius angustus_16.jpg',
 'Othius/Othius angustus_17.jpg',
 'Othius/Othius angustus_18.jpg',
 'Othius/Othius angustus_19.jpg',
 'Othius/Othius angustus_20.jpg',
 'Othius/Othius angustus_21.jpg',
 'Othius/Othius angustus_22.jpg',
 'Othius/Othius angustus_23.jpg',
 'Othius/Othius angustus_24.jpg',
 'Othius/Othius angustus_25.jpg',
 'Othius/Othius angustus_26.jpg',
 'Othius/Othius angustus_9.jpg',
 'Othius/Othius punctulatus_10.jpg',
 'Othius/Othius punctulatus_11.jpg',
 'Othius/Othius punctulatus_12.jpg',
 'Othius/Othius punctulatus_13.jpg',
 'Othius/Othius punctulatus_14.jpg',
 'Othius/Othius punctulatus_15.jpg',
 'Othius/Othius punctulatus_5.jpg',
 'Othius/Othius punctulatus_6.jpg',
 'Othius/Othius punctulatus_7.jpg',
 'Othius/Othius punctulatus_8.jpg',
 'Othius/Othius punctul

In [10]:
val_generator.classes

array([ 0,  0,  0, ..., 43, 43, 43], dtype=int32)

In [11]:
labels[0]

'Othius'

In [41]:
d = "./val_images/"
im_names = val_generator.filenames
for filename in sorted(os.listdir(d)):
    index = int(filename.split("_")[1])
    name = im_names[index].split("/")[-1]
    newname = filename.split("_")[1]+"_"+name
    os.rename(d + filename, d + newname)

In [30]:
Y_pred = model.predict_generator(val_generator, steps=validation_steps)

In [31]:
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print(classification_report(val_generator.classes, y_pred, target_names=labels))

In [ ]:
y_true = val_generator.classes
print(y_true)
matplotlib.rcParams['figure.figsize'] = [10, 10]
plot_confusion_matrix(y_true, y_pred, np.array(labels))